In [32]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import warnings
warnings.filterwarnings("ignore")
import re

In [7]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 6)  # Adjust the number of output neurons for 123 classes
        self.log_softmax = nn.LogSoftmax(dim=1)  # Use dim=1 for LogSoftmax in classification

    def forward(self, sent_id, mask):
        sent_id = torch.tensor(sent_id)
        outputs = self.bert(input_ids=sent_id, attention_mask=mask)
        last_hidden_state_cls = outputs[0][:, 0, :]
        x = self.fc1(last_hidden_state_cls)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.log_softmax(x)
        return x

In [33]:
def get_output(text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    my_dict = {
            0:'Sad', 1:'Happy', 2:'Happy', 3: 'Sad', 4: 'Fear', 5:'Suprise'
    }
    
    def NLP_cleaning(text):
        sent = text
        sent = re.sub('<[^>]*>', '', sent)
        sent = re.sub('[^a-zA-z0-9]', ' ', sent)
        sent = sent.lower()
        return sent
    
    def load_checkpoint(filepath):
        checkpoint = torch.load(filepath, map_location=torch.device('cpu'))
        model = checkpoint['model']
        model.load_state_dict(checkpoint['state_dict'])
        for parameter in model.parameters():
            parameter.requires_grad = False
        model.eval()
        return model
    
    model = load_checkpoint('checkpoint.pth')
    device = 'cpu'
    
    def Predict(text):
        encoded_review = tokenizer.encode_plus(
          text,
          max_length=256,
          truncation = True,
          add_special_tokens=True,
          return_token_type_ids=False,
          padding = 'max_length',
          return_attention_mask=True,
          return_tensors='pt',
        )

        input_ids = encoded_review['input_ids'].to(device)
        attention_mask = encoded_review['attention_mask'].to(device)
        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output, dim=1)
        extracted_value = prediction.item()
        return extracted_value
    
    preprocessed = NLP_cleaning(text)
    output = Predict(preprocessed)
    return my_dict[output]

In [40]:
ans = get_output("i hate life")
ans

'Sad'